In [1]:
import os
cwd = os.getcwd()
cwd
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
import pandas as pd 
from glob import glob
import PIL.Image as pil 
import PIL.ImageOps
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
CSV = pd.read_csv("D:/Udacity/Sub3_dog_breed/Xrayimages_004/Data_Entry_2017/Data_Entry_2017.csv")
sample=CSV[['Image Index','Finding Labels']]



In [3]:
#parsing file 
sample['Finding Labels'] = sample['Finding Labels'].apply(lambda x: x.split('|')[0])

sample.head()

len(sample)

a=set(sample['Finding Labels'])

print('The total number of disease categories  is = %d ' % len(a))

sample['Finding Labels'] = sample['Finding Labels'].apply(lambda x: x.split('|')[0])


The total number of disease categories  is = 15 


C:\Users\mahmo\Anaconda3\envs\dog-project\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\mahmo\Anaconda3\envs\dog-project\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
sample.head()

,Image Index,Finding Labels
0,00000001_000.png,Cardiomegaly
1,00000001_001.png,Cardiomegaly
2,00000001_002.png,Cardiomegaly
3,00000002_000.png,No Finding
4,00000003_000.png,Hernia


In [5]:
sample.count()

Image Index       112120
Finding Labels    112120
dtype: int64

# Read File Names

In [6]:
filespathTrain="D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages"


In [7]:
filespathValid="D:/Udacity/Sub3_dog_breed/Xrayimages_001/RGBValidationimages"


In [8]:
def get_lst_image_names(file_path):
    """
    Reads in all files from file path into a list.
    INPUT
        file_path: specified file path containing the images.
    OUTPUT
        List of image strings
    """
    list_img_names=[i for i in os.listdir(file_path) if i != '.DS_Store']
    return pd.DataFrame({'Image Index' :list_img_names})




In [9]:
list_img_names_train=get_lst_image_names(filespathTrain)

#len(list_img_names)
#Image_index_Df = pd.DataFrame({'Image Index' :list_img_names})

list_img_names_train.head()


,Image Index
0,00006585_007.png
1,00006585_008.png
2,00006585_009.png
3,00006585_010.png
4,00006585_011.png


In [10]:
list_img_names_valid=get_lst_image_names(filespathValid)

#len(list_img_names)
#Image_index_Df = pd.DataFrame({'Image Index' :list_img_names})

list_img_names_valid.head()


,Image Index
0,00000001_000.png
1,00000001_001.png
2,00000001_002.png
3,00000002_000.png
4,00000003_000.png


# Read CSV and marge with File Names

In [11]:
def mergfun(Image_index_Df):
    Image_labelsDf = pd.merge(Image_index_Df, sample, how='left', on='Image Index')
    return Image_labelsDf



In [12]:
Image_labelsTrainDf=mergfun(list_img_names_train)
Image_labelsTrainDf.head()


,Image Index,Finding Labels
0,00006585_007.png,Atelectasis
1,00006585_008.png,No Finding
2,00006585_009.png,Infiltration
3,00006585_010.png,Infiltration
4,00006585_011.png,Infiltration


In [13]:
Image_labelsTrainDf.count()

Image Index       10000
Finding Labels    10000
dtype: int64

In [14]:
Image_labelsValidDf=mergfun(list_img_names_valid)
Image_labelsValidDf.head()

,Image Index,Finding Labels
0,00000001_000.png,Cardiomegaly
1,00000001_001.png,Cardiomegaly
2,00000001_002.png,Cardiomegaly
3,00000002_000.png,No Finding
4,00000003_000.png,Hernia


In [15]:
Image_labelsValidDf.count()

Image Index       4999
Finding Labels    4999
dtype: int64

# Preprocess CSV file

# To find one hot vector for single Lable

In [17]:
def singleLabel(mDf):
    #dfparsed1=pd.concat([mDf[['Image Index']], mDf['Finding Labels'].str.split('|').str.join('')],axis=1)
    dfparsed1=pd.concat([mDf[['Image Index']], pd.get_dummies(mDf['Finding Labels'])],axis=1)
    return dfparsed1
    


In [18]:
singleLableDfTrain=singleLabel(Image_labelsTrainDf) 
#singleLableDf.count()
singleLableDfTrain.head(5)
tranTarget1= singleLableDfTrain.drop('Image Index', 1)
tranTarget=tranTarget1.values
#singleLableDf.count()

In [19]:
tranTarget.shape

(10000, 15)

In [20]:
singleLableDfValid=singleLabel(Image_labelsValidDf) 

singleLableDfValid.head(2)
validTarget1 = singleLableDfValid.drop('Image Index', 1)
validTarget=validTarget1.values
#singleLableDf.count()

In [21]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


In [22]:
def prepareListOfImageName(listOfFlile, pathRGB): 
    listOfFlile=listOfFlile['Image Index']
    #listOfFlile[0:10]
    l=[]
    for filename in listOfFlile:
        l.append(pathRGB+'/'+filename) 
        #l.append('D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/'+filename)
    return np.array(l)    
   #trainPaths = np.array(l)

In [23]:
trainImgeNameArray=prepareListOfImageName(singleLableDfTrain, filespathTrain)

In [25]:
trainImgeNameArray[0:5]

array(['D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00006585_007.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00006585_008.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00006585_009.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00006585_010.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_004/RGBimages/00006585_011.png'], 
      dtype='<U67')

In [26]:
validImgeNameArray=prepareListOfImageName(singleLableDfValid,filespathValid)

In [27]:
validImgeNameArray[0:5]

array([ 'D:/Udacity/Sub3_dog_breed/Xrayimages_001/RGBValidationimages/00000001_000.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_001/RGBValidationimages/00000001_001.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_001/RGBValidationimages/00000001_002.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_001/RGBValidationimages/00000002_000.png',
       'D:/Udacity/Sub3_dog_breed/Xrayimages_001/RGBValidationimages/00000003_000.png'], 
      dtype='<U77')

In [28]:
train_tensors = paths_to_tensor(trainImgeNameArray).astype('float32')/255;

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:59<00:00, 168.03it/s]


In [29]:
valid_tensors = paths_to_tensor(validImgeNameArray).astype('float32')/255;

100%|█████████████████████████████████████████████████████████████████████████████| 4999/4999 [00:44<00:00, 111.36it/s]


In [30]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
print(len(train_tensors))
print(train_tensors.shape)
#import random
#random.seed(8675309)

model = Sequential()

### Define architecture.

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu',input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(GlobalAveragePooling2D())
model.add(Dropout(0.3))

model.add(Dense(15, activation='softmax'))
model.summary()

10000
(10000, 224, 224, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 64)  

In [31]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint  

###  specify the number of epochs that you would like to use to train the model.

epochs = 10

###   the code below set the chechpoints and save the model

#checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, tranTarget, 
          validation_data=(valid_tensors, validTarget),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

In [32]:
epochs = 10
model.fit(train_tensors, tranTarget, 
          validation_data=(valid_tensors, validTarget),
          epochs=epochs, batch_size=20,  verbose=1)

Train on 10000 samples, validate on 4999 samples
Epoch 1/10
10000/10000 [==============================] - ETA: 491s - loss: 2.5350 - acc: 0.1500 - ETA: 398s - loss: 2.2316 - acc: 0.3750 - ETA: 378s - loss: 2.2594 - acc: 0.3833 - ETA: 363s - loss: 2.1335 - acc: 0.4375 - ETA: 353s - loss: 2.1121 - acc: 0.4400 - ETA: 339s - loss: 2.1202 - acc: 0.4250 - ETA: 326s - loss: 2.1435 - acc: 0.4214 - ETA: 318s -